In [2]:
!pip install plotly
!pip install pandas
!pip install requests
!pip install geopy

In [10]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import requests
import plotly.io as pio

# Load the dataset into a DataFrame
data = pd.read_csv('covidnewpractice.csv')

# Convert date column to datetime
data['date'] = pd.to_datetime(data['date'])

# Define a function to geocode a country name
def geocode_country(country):
    base_url = "https://nominatim.openstreetmap.org/search"
    params = {
        "q": country,
        "format": "json",
        "limit": 1
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data:
            latitude = float(data[0]['lat'])
            longitude = float(data[0]['lon'])
            return latitude, longitude
    return None, None

# Create a figure
fig = go.Figure()

# Iterate over unique recorded dates
for date in data['date'].dt.date.unique():
    # Filter data for the specific date
    filtered_data = data[data['date'].dt.date == date]

    # Convert country names to latitude and longitude coordinates
    latitude, longitude = zip(*filtered_data['location'].apply(geocode_country))
    data.loc[filtered_data.index, 'latitude'] = latitude
    data.loc[filtered_data.index, 'longitude'] = longitude

    # Drop rows with missing coordinates
    filtered_data = filtered_data.dropna(subset=['latitude', 'longitude'])

    # Add a heatmap trace for the date
    fig.add_trace(go.Densitymapbox(
        lat=filtered_data['latitude'],
        lon=filtered_data['longitude'],
        z=filtered_data['new_cases_smoothed'],  # Your value column for color intensity
        radius=10,
        coloraxis="coloraxis"
    ))

# Set up the layout
fig.update_layout(
    mapbox_style="open-street-map",
    mapbox_center_lon=0,
    mapbox_center_lat=0,
    mapbox_zoom=0,
    coloraxis=dict(colorscale='Viridis', cmin=0, cmax=data['new_cases_smoothed'].max())
)

# Create a slider to control the date
slider = dict(
    active=0,
    steps=[go.layout.slider.Step(method='restyle', args=[{'visible': [True if i == j else False for i in range(len(fig.data))]}],
                                 label=str(date.date()), value=str(date.date()))
           for j, date in enumerate(data['date'].dt.date.unique())]
)

# Update the layout with the slider
fig.update_layout(sliders=[slider])

# Export the figure as an HTML file
pio.write_html(fig, 'recent_covid_heatmap.html')


KeyError: ['latitude', 'longitude']